In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import missingno as msno
import pandas_profiling
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
#解决中文显示问题
plt.rcParams["font.sans-serif"] = [u"SimHei"]
plt.rcParams["axes.unicode_minus"] = False
#解决图中负号问题
plt.rcParams['axes.unicode_minus']=False

In [130]:
column_name=['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg','os','battery_capacity','storage','screen_size']

# names = ['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg']
original = pd.read_table('ZZ0702GuangFa_Samples100000_FeatureData.txt',names=column_name)
display(original.head(),original.shape)

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,13013302938,1,0.0,NaN,联通联通,9.0,0.0,0.0,0.0,0.0,...,NaN,NaN,38.863332,0.0,NaN,0.0,4.0,4120mAh,64GB,6.0
1,13022062073,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13035901978,1,NaN,NaN,NaN,37.2,0.0,0.0,0.0,0.0,...,NaN,NaN,37.013334,0.0,NaN,0.0,NaN,NaN,NaN,NaN
3,13042151031,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3050mAh,16/64GB,5.0
4,13045173577,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(103513, 36)

In [131]:
display(original.target.value_counts())

0    92989
1    10524
Name: target, dtype: int64

In [132]:
sample_false=original[original['target']==0].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
sample_true=original[original['target']==1].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
combin_df=sample_true.append(sample_false)
combin_df.to_excel('d:\sample_model.xlsx',index=None)

In [133]:
sample_true.head()

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.48,29.0,NaN,NaN,NaN,NaN
1,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.0,4.0,2920mAH,16GB/32GB,5.0
2,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.20,135.0,4.0,3000mAh,32GB,5.5
3,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.00,90.0,4.0,3000mAh,16GB,5.7
4,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.0,3.0,1715mAh,16GB/64GB/128GB,4.7


In [34]:
import random
import pandas as pd
def create_column_randomsample(column_random_dict):
    total_column_dict={}
    total_column_df=[]
    for columns_name in column_random_dict:
        if columns_name not in total_column_dict:
            total_column_dict[columns_name]=[]        
        for value_keys in column_random_dict[columns_name]:
            value_keys_counts=int(column_random_dict[columns_name][value_keys]*50000)
            for index in range(value_keys_counts):
                total_column_dict[columns_name].append(value_keys)
            
    result_df=pd.DataFrame(total_column_dict)
    for columns in result_df.columns:
        result_df[columns]=result_df[columns].sample(frac=1).reset_index(drop=True)
    return result_df

def create_number_randomsample(number_random_dict):
    import numpy as np
    import random
    
    total_column_dict={}
    
    all_True_column_df={}
    all_False_column_df={}
    
    for columns_name in number_random_dict:
        
        if columns_name not in total_column_dict:
            signle_True_column_df=[]
            for column_section_key in number_random_dict[columns_name]:
                True_column_df=[]
                
                left_value=int(column_section_key.split('-')[0])
                right_value=int(column_section_key.split('-')[1])
                
                True_sample_number=int(100000*number_random_dict[columns_name][column_section_key]['out_ratio']*number_random_dict[columns_name][column_section_key]['in_ratio_true'])
                for value_number in range(True_sample_number):
                    True_column_df.append(random.randint(left_value,right_value))
                #print(left_value,right_value,True_column_df)
                signle_True_column_df+=(True_column_df)
            #print(signle_True_column_df)
            all_True_column_df[columns_name]=signle_True_column_df
            
    
    for columns_name in number_random_dict:
        
        if columns_name not in total_column_dict:
            signle_False_column_df=[]
            for column_section_key in number_random_dict[columns_name]:
                False_column_df=[]
                
                left_value=int(column_section_key.split('-')[0])
                right_value=int(column_section_key.split('-')[1])
                False_sample_number=int(100000*number_random_dict[columns_name][column_section_key]['out_ratio'])-int(100000*number_random_dict[columns_name][column_section_key]['out_ratio']*number_random_dict[columns_name][column_section_key]['in_ratio_true'])
                for value_number in range(False_sample_number):
                    False_column_df.append(random.randint(left_value,right_value))
                #print(left_value,right_value,True_column_df)
                signle_False_column_df+=(False_column_df)
            all_False_column_df[columns_name]=signle_False_column_df
            
    for columns_name in number_random_dict:       
        total_column_dict[columns_name]=all_True_column_df[columns_name]+all_False_column_df[columns_name]
        
    
    return pd.DataFrame(total_column_dict)
            
    
True_number_random_dict={
                        '年龄':{
                                 '0-17':{'out_ratio':0.01,'in_ratio_true':0.3},'18-25':{'out_ratio':0.06,'in_ratio_true':0.4},
                                 '25-35':{'out_ratio':0.13,'in_ratio_true':0.6},'35-45':{'out_ratio':0.2,'in_ratio_true':0.7},
                                 '45-60':{'out_ratio':0.27,'in_ratio_true':0.6},'60-70':{'out_ratio':0.23,'in_ratio_true':0.5},
                                 '70-120':{'out_ratio':0.1,'in_ratio_true':0.4},}
                        }



True_column_random_dict={'性别':{'男':0.6,'女':0.4},'婚姻状况':{'已婚':0.65,'未婚':0.35},
                         '学历':{'小学':0.01,'初中':0.02,'高中':0.1,'初中中专':0.02,'高中中专':0.1,'大专':0.5,'本科':0.2,'研究生':0.05},
                         '征信认定年收入':{'5万':0.35,'10万':0.5,'20万':0.05,'30万':0.04,'40万':0.03,'50万':0.02,'大于50万':0.01},
                         '社会保障':{'1':0.7,'0':0.3},                         
                         '单位性质':{'国有企业':0.14,'私营企业':0.26,'股份制企业':0.3,'外商投资企业':0.3},
                         '客户评级结果':{'C1':0.15,'C2':0.15,'C3':0.3,'C4':0.2,'C5':0.15,'C6':0.05},
                         '手机品牌':{'苹果':0.3,'华为':0.2,'荣耀':0.1,'小米':0.1,'OPPO':0.16,'vivo':0.14},
                         '客户风险承受能力等级':{'较低':0.2,'基本':0.45,'中等':0.3,'较高':0.05},                
                         '客户级信用额度':{'5000':0.05,'10000':0.15,'20000':0.3,'30000':0.3,'50000':0.2},
                         '零售潜在客户':{'是':0.6,'否':0.4},
                         '近12月是否贷款':{'是':0.32,'否':0.68},                         
                        }

False_column_random_dict={'性别':{'男':0.45,'女':0.55},'婚姻状况':{'已婚':0.4,'未婚':0.6},
                         '学历':{'小学':0.02,'初中':0.03,'高中':0.1,'初中中专':0.1,'高中中专':0.4,'大专':0.3,'本科':0.03,'研究生':0.02},
                         '征信认定年收入':{'5万':0.55,'10万':0.35,'20万':0.04,'30万':0.03,'40万':0.02,'50万':0.008,'大于50万':0.002},
                         '社会保障':{'1':0.6,'0':0.4},
                         '单位性质':{'国有企业':0.14,'私营企业':0.26,'股份制企业':0.3,'外商投资企业':0.3},
                         '客户评级结果':{'C1':0.2,'C2':0.45,'C3':0.2,'C4':0.1,'C5':0.03,'C6':0.02},
                         '手机品牌':{'苹果':0.05,'华为':0.1,'荣耀':0.25,'小米':0.4,'OPPO':0.1,'vivo':0.1},
                         '客户风险承受能力等级':{'较低':0.5,'基本':0.35,'中等':0.12,'较高':0.03},                       
                         '客户级信用额度':{'5000':0.1,'10000':0.4,'20000':0.2,'30000':0.2,'50000':0.1},
                         '零售潜在客户':{'是':0.3,'否':0.7},
                         '近12月是否贷款':{'是':0.35,'否':0.65},
                        }


total_df=create_number_randomsample(True_number_random_dict)

In [36]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 1 columns):
年龄    100000 non-null int64
dtypes: int64(1)
memory usage: 781.3 KB


In [35]:
pandas_profiling.ProfileReport(total_df)

Number of variables,1
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,781.3 KiB
Average record size in memory,8.0 B
Numeric,1
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [ ]:
                         
'''
true_result_df=create_column_randomsample(True_column_random_dict)
true_result_number_df=create_number_randomsample(True_number_random_dict)

True_combin_df=true_result_df.merge(true_result_number_df,left_index=True, right_index=True)
'''


#True_combin_df=True_combin_df.merge(pd.DataFrame(sample_true[['id','owner_city']]),left_index=True, right_index=True)
#True_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)

                            
                                
'''
True_combin_df['target']=1


False_result_df=create_column_randomsample(False_column_random_dict)
False_result_number_df=create_number_randomsample(False_number_random_dict)
False_combin_df=False_result_df.merge(False_result_number_df,left_index=True, right_index=True)
#False_combin_df=False_combin_df.merge(pd.DataFrame(sample_false[['id','owner_city']]),left_index=True, right_index=True)
#False_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)
False_combin_df['target']=0

total_comple=True_combin_df.append(False_combin_df).reset_index(drop=True)

#total_comple=total_comple[['id','性别','年龄','籍贯','婚姻状况','学历','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','附卡卡数','存款期限','是否为低保户','受否贫困户','行业','职业','零售潜在客户','理财产品风险等级','手机品牌','基金风险等级','理财产品是否保本','客户风险承受能力等级','电子渠道购买标志','贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债','target']]
total_comple=total_comple[['性别','年龄','婚姻状况','学历','开户日期','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','附卡卡数','存款期限','是否为低保户','受否贫困户','行业','职业','零售潜在客户','理财产品风险等级','手机品牌','基金风险等级','理财产品是否保本','客户风险承受能力等级','电子渠道购买标志','贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债','target']]


# 生成随机日期函数：
import datetime
def create_random_date(rowdata):
    #print (datetime.datetime.now()+datetime.timedelta(days=-1)).strftime("%Y-%m-%d %H:%M:%S")
    #print(datetime.datetime.now()+datetime.timedelta(days=rowdata['开户日期']).strftime("%Y-%m-%d"))
    print((datetime.datetime.now()-datetime.timedelta(days=rowdata['开户日期'])).strftime("%Y-%m-%d"))
    return (datetime.datetime.now()-datetime.timedelta(days=rowdata['开户日期'])).strftime("%Y-%m-%d")
total_comple['开户日期']=total_comple.apply(create_random_date,axis=1)
'''

In [135]:
total_comple.head()

,性别,年龄,婚姻状况,学历,开户日期,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,...,结构性存款_人民币_资产负债,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,target
0,男,12,未婚,大专,1999-08-15,5万,0,20000,0,核心,...,49405,23432,25468,10041,4184,78639,376452,12277,5605,1
1,女,25,已婚,初中中专,2019-09-17,10万,1,50000,0,普通,...,39884,18805,14512,7880,3957,80815,379467,11764,3858,1
2,男,-10,未婚,大专,2015-03-27,5万,0,10000,0,普通,...,50171,11834,11639,9109,4451,66963,295913,18028,5434,1
3,男,20,已婚,大专,2002-10-13,10万,1,30000,0,核心,...,44882,19946,9732,10432,4529,70032,510295,14891,4267,1
4,男,19,已婚,大专,2003-02-18,5万,0,50000,0,普通,...,44677,15478,12109,9432,4324,72221,343757,18678,5915,1


In [136]:
# 对样本数据进行处理，将负值修改为0

random_columns_list=['贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债']


def deal_zero_value(rowdata,column):
    if rowdata[column]<0:
        return 0
    else:
        return rowdata[column]
    
    
for random_colum in random_columns_list:    
    total_comple[random_colum]=total_comple.apply(deal_zero_value,axis=1, args = (random_colum,))


In [137]:
display(True_combin_df.head(),False_combin_df.head(),total_comple.tail())
print(total_comple.shape,True_combin_df.shape,False_combin_df.shape,total_comple.columns)

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,开户日期,target
0,男,未婚,大专,5万,0,20000,0,核心,C5,金卡,...,23432,25468,10041,4184,78639,376452,12277,5605,7446.714355,1
1,女,已婚,初中中专,10万,1,50000,0,普通,C1,金卡,...,18805,14512,7880,3957,80815,379467,11764,3858,108.866684,1
2,男,未婚,大专,5万,0,10000,0,普通,C3,普卡,...,11834,11639,9109,4451,66963,295913,18028,5434,1744.237549,1
3,男,已婚,大专,10万,1,30000,0,核心,C2,白金,...,19946,9732,10432,4529,70032,510295,14891,4267,6292.313477,1
4,男,已婚,大专,5万,0,50000,0,普通,C4,普卡,...,15478,12109,9432,4324,72221,343757,18678,5915,6164.348145,1


,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,开户日期,target
0,男,已婚,小学,40万,1,10000,0,普通,C3,普卡,...,11747,9048,9531,929,47535,213117,6595,22222,1088.429199,0
1,女,未婚,大专,5万,1,10000,0,准核心,C2,白金,...,7289,12950,8118,861,48017,162023,11036,9673,1220.908081,0
2,女,已婚,高中中专,5万,0,20000,0,普通,C4,普卡,...,15520,9893,7520,767,53450,151895,10159,-3269,957.995117,0
3,男,未婚,高中中专,5万,1,10000,0,普通,C3,普卡,...,8076,9410,4284,487,45945,103304,6625,-11558,1077.512817,0
4,女,未婚,高中中专,5万,1,10000,0,普通,C3,白金,...,13992,13881,4240,786,55331,176267,10152,-4469,689.687500,0


,性别,年龄,婚姻状况,学历,开户日期,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,...,结构性存款_人民币_资产负债,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,target
99995,男,29,已婚,高中中专,2017-03-30,10万,1,20000,0,普通,...,41915,15396,9325,5536,906,50827,82965,11304,9481,0
99996,男,37,未婚,高中,2017-10-11,10万,1,5000,0,核心,...,25032,24468,12483,5174,1037,48866,183879,8211,0,0
99997,男,25,未婚,高中中专,2017-03-30,10万,0,10000,0,普通,...,27005,18522,9303,6758,721,54899,182742,10197,8090,0
99998,女,29,未婚,高中中专,2016-12-19,10万,1,50000,0,普通,...,45068,12262,5797,4672,1036,52021,243886,9382,613,0
99999,男,9,未婚,大专,2017-05-14,5万,0,5000,0,准核心,...,35195,13049,12545,4444,1359,47716,170418,9210,19285,0


(100000, 63) (50000, 63) (50000, 63) Index(['性别', '年龄', '婚姻状况', '学历', '开户日期', '征信认定年收入', '社会保障', '客户级信用额度',
       '美元账户额度', 'CRM评级结果', '客户评级结果', '借记卡层级', '附卡卡数', '存款期限', '是否为低保户',
       '受否贫困户', '行业', '职业', '零售潜在客户', '理财产品风险等级', '手机品牌', '基金风险等级', '理财产品是否保本',
       '客户风险承受能力等级', '电子渠道购买标志', '贷款放款本金', '账户余额', '贷款期数', '剩余贷款本金',
       '购买金额_事件表', '数量_事件表', '可用余额_事件表', '分期金额_事件表', '贷款期限_事件表', '未还本金_事件表',
       '偿还金额_事件表', '金额_事件表', '交易金额_交易日志表', '产品万份收益_基金产品', '车产价值_字典列表',
       '房产价值_字典列表', '交易金额_交易往来', '存期_交易账务', '交易金额_交易账务', '行内总资产_资产负债',
       '行内总负债_资产负债', '本外币存款_资产负债', '人民币活期存款_资产负债', '人民币定期存款_资产负债',
       '本外币理财_资产负债', '人民币理财_资产负债', '万利宝理财_资产负债', '保本型理财_人民币_资产负债',
       '结构性存款_人民币_资产负债', '兴业宝资产_资产负债', '综合金融资产_资产负债', '券商集合理财_资产负债', '保险_资产负债',
       '个人贷款_资产负债', '个人购房按揭贷款_资产负债', '个人委托贷款_资产负债', '个人消费贷款_资产负债', 'target'],
      dtype='object')


In [138]:
total_comple.to_excel('d:/上海兴业银行样本数据集.xlsx',index=None)


In [ ]:
# 生成随机日期函数：
import
def create_random_date(rowdata):
    print(rowdata.)
    

In [8]:
True_combin_df.head()

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,存款期限,是否为低保户,受否贫困户,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金,id,籍贯
0,女,未婚,本科,5万,1,20000,小于0,核心,C4,普卡,...,6个月,否,否,10,539739,-7114,123,404910,13013302938,联通联通
1,男,已婚,大专,20万,1,20000,大于0,核心,C4,金卡,...,1年,否,否,48,807197,74019,63,404393,13022062073,NaN
2,男,已婚,高中中专,5万,1,10000,小于0,核心,C6,白金,...,1年,否,否,11,679981,216705,117,63766,13035901978,NaN
3,男,已婚,高中,5万,1,50000,小于0,核心,C1,白金,...,5年,否,否,30,498424,185155,201,330451,13042151031,NaN
4,男,已婚,大专,10万,1,50000,小于0,核心,C1,普卡,...,5年,否,否,12,511549,115947,-6,326409,13045173577,NaN


In [21]:
combin_df[['id','性别','籍贯','婚姻状况','学历','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','账户余额','剩余贷款本金']].to_excel('d:\上海兴业银行样本造数.xlsx',index=None)

In [164]:
display(true_result_df.head(),true_result_number_df.head())

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,附卡卡数,存款期限,是否为低保户,受否贫困户
0,男,已婚,大专,5万,1,30000,小于0,普通,C3,黑金,4,1年,否,否
1,男,已婚,高中,10万,1,30000,小于0,准核心,C3,普卡,0,3个月,否,否
2,女,已婚,本科,5万,0,20000,小于0,普通,C4,普卡,3,1年,否,否
3,男,已婚,大专,10万,1,20000,小于0,普通,C3,金卡,1,2年,否,否
4,女,已婚,小学,5万,1,20000,小于0,普通,C3,白金,0,5年,否,否


,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金
0,37,691320,8,317,445082
1,15,645061,29,49,196353
2,19,579867,29,157,368844
3,10,667989,34,33,329207
4,30,566405,51,118,330431


In [12]:
pd.DataFrame

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
97032,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.480000,29.000000,NaN,NaN,NaN,NaN
29750,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.000000,4.0,2920mAH,16GB/32GB,5.0
36394,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.200000,135.000000,4.0,3000mAh,32GB,5.5
79629,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.000000,90.000000,4.0,3000mAh,16GB,5.7
40228,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.000000,3.0,1715mAh,16GB/64GB/128GB,4.7
82230,15980726225,1,1.0,NaN,福建福州,78.00,0.00,62.0,0.0,0.0,...,35.0,NaN,91.333333,0.0,NaN,75.333333,4.0,2300mAh,16GB,5.0
62550,15217163366,1,2.0,50.0,广东珠海,55.12,0.00,28.0,0.0,0.0,...,NaN,NaN,54.293334,0.0,NaN,28.000000,4.0,3100mAh,8GB,5.5
99664,18811186422,1,1.0,43.0,北京北京,118.00,0.00,118.0,0.0,0.0,...,NaN,NaN,118.224998,0.0,NaN,118.000000,4.0,3000mAh,16GB,5.5
93368,18374069022,1,0.0,44.0,湖南株洲,31.49,0.00,0.0,0.0,0.0,...,NaN,NaN,22.496667,0.0,NaN,0.000000,NaN,NaN,NaN,NaN
13231,13580829297,1,0.0,NaN,广东东莞,178.00,0.00,173.0,0.0,0.0,...,NaN,NaN,176.866669,0.0,NaN,171.333333,4.0,3750mAh,64GB/128GB/256GB,5.5
